### This is a trivial example notebook for Document Field Extraction
This is a very simple solution using an LLM.

In [7]:
import base64
from IPython.display import display, HTML

import instructor
from openai import OpenAI
from pydantic import BaseModel

### Lets check out a document that we can extract some fields from

In [8]:
img1 = "example_docs/img1.png"

html_code = f"""
<div style="width: 100%; overflow-x: auto; white-space: nowrap;">
    <img src="{img1}" style="width: 800px">
</div>
"""
display(HTML(html_code))

### Setup a client and call ChatGPT

In [11]:
client = instructor.from_openai(OpenAI())

In [12]:
class FieldExtraction(BaseModel):
    customer_name: str
    customer_address: str
    products_sold: list
    price: float

In [13]:
with open(img1, "rb") as f:
    encoded_file = base64.b64encode(f.read()).decode("utf-8")
completion = client.chat.completions.create(
     model="gpt-4o",
     response_model=FieldExtraction,
     messages=[
        {"role": "system", "content": "Please extract relevant values based on the provided keys."},
        {"role": "user", "content": {"type": "image_url", "image_url": {"url": f"data:image/png;base64,{encoded_file}"}}},
    ],
)


### Check results

In [30]:
for k,v in completion.model_dump().items():
    print(f"{k}: {v}")

customer_name: John Smith
customer_address: 16040 S. US 27
Lansing, Michigan 48906
products_sold: ['Brake Discs, Pads & Calipers', 'Control Arm', 'Suspension Lift Kit']
price: 2694.3
